In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
d={
    'hash'          :'category',
    'trajectory_id' :'category',
    'time_entry'    :'category',
    'time_exit'    :'category',
    'vmax':'float64',
    'vmin':'float64',
    'vmean':'float64',
    'x_entry':'float64',
    'y_entry':'float64',
    'x_exit':'float64',
    'y_exit':'float64'
}

train = pd.read_csv('./input/data_train.csv',nrows=10000,dtype=d).drop(columns=['Unnamed: 0'])
test = pd.read_csv('./input/data_test.csv',nrows=10000,dtype=d).drop(columns=['Unnamed: 0'])

In [3]:
def direct(ax, ay, bx, by):
    return bx-ax, by-ay


def calc_directory(v):
    # 7,8 ->9,10 
    return direct(v[7],v[8],v[9],v[10])


def add_directories(dirx, diry, indexes, df):
    first = True
    meanx, meany = 0, 0
    for i in indexes:
        if first==True:
            first = False
            dirx[i], diry[i] = calc_directory(df.values[i])
            
        else:    
            x, y = calc_directory(df.values[last])
            meanx, meany = (x+meanx)/2, (y+meany)/2
            dirx[i], diry[i] = meanx, meany
            
        last = i
        # print(dirx[i], diry[i])
      
        
def make_directories(df):
    dirx=np.empty(len(df))
    diry=np.empty(len(df))
    drivers = df.hash.unique()
    """
    for driver in drivers[:100]:
        indexes= train[train.hash == driver].index
        add_directories(dir, indexes, df)
    """
    for driver in drivers:
        add_directories(dirx, diry, df[df.hash == driver].index, df) 
    
    df['x_dir'] = dirx
    df['y_dir'] = diry

    
make_directories(train)
make_directories(test)

In [4]:
def find_last_trajectories(df):
    drivers = df.hash.unique()
    df2 = pd.DataFrame(columns=df.columns)
    for driver in drivers:
        df2 = df2.append(df[df.hash == driver][-1:])
    df2 = df2.reset_index(drop=True)    
    df2.to_csv('./input/test2.csv',index=False)
       

find_last_trajectories(test)

test2 = pd.read_csv('./input/test2.csv')
trajectory_id = test2['trajectory_id']

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
hash             10000 non-null category
trajectory_id    10000 non-null category
time_entry       10000 non-null category
time_exit        10000 non-null category
vmax             3013 non-null float64
vmin             3013 non-null float64
vmean            3174 non-null float64
x_entry          10000 non-null float64
y_entry          10000 non-null float64
x_exit           10000 non-null float64
y_exit           10000 non-null float64
x_dir            10000 non-null float64
y_dir            10000 non-null float64
dtypes: category(4), float64(9)
memory usage: 2.0 MB


In [6]:
train[:10]

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,x_dir,y_dir
0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,07:04:31,07:08:32,NaN,NaN,NaN,3.751014e+06,-1.909398e+07,3.750326e+06,-1.913634e+07,-687.954453,-42359.276973
1,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_1,07:20:34,07:25:42,NaN,NaN,NaN,3.743937e+06,-1.932247e+07,3.744975e+06,-1.931966e+07,-343.977227,-21179.638486
2,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_2,07:53:32,08:03:25,NaN,NaN,NaN,3.744868e+06,-1.929356e+07,3.744816e+06,-1.929284e+07,346.760214,-9187.894884
3,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_3,08:17:50,08:37:23,NaN,NaN,NaN,3.744880e+06,-1.929229e+07,3.744809e+06,-1.929049e+07,147.220027,-4232.062464
4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,14:38:09,14:38:09,NaN,NaN,NaN,3.744909e+06,-1.928558e+07,3.744909e+06,-1.928558e+07,37.987776,-1214.446263
5,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_5,15:02:31,15:18:33,NaN,NaN,NaN,3.744945e+06,-1.928183e+07,3.744785e+06,-1.928148e+07,18.993888,-607.223131
6,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_0,14:29:23,14:29:23,0.000000,0.000000,0.000000,3.749450e+06,-1.926506e+07,3.749450e+06,-1.926506e+07,0.000000,0.000000
7,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_1,14:39:49,14:39:49,0.000000,0.000000,0.000000,3.749090e+06,-1.926726e+07,3.749090e+06,-1.926726e+07,0.000000,0.000000
8,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_2,14:50:14,14:50:14,0.507809,0.507809,0.507809,3.749042e+06,-1.926632e+07,3.749042e+06,-1.926632e+07,0.000000,0.000000
9,0000cf177130469eeac79f67b6bcf3df_9,traj_0000cf177130469eeac79f67b6bcf3df_9_3,15:00:32,15:29:48,1.149404,1.149404,1.149404,3.749088e+06,-1.926605e+07,3.749610e+06,-1.926594e+07,0.000000,0.000000


In [7]:
test[:10]

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,x_dir,y_dir
0,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_0,11:43:17,11:50:17,NaN,NaN,NaN,3.773413e+06,-1.909828e+07,3.773111e+06,-1.914508e+07,-302.789015,-46798.074140
1,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_2,12:21:37,12:21:37,0.00,0.00,0.00,3.773199e+06,-1.914354e+07,3.773199e+06,-1.914354e+07,-151.394507,-23399.037070
2,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_3,12:34:27,13:14:11,NaN,NaN,NaN,3.763760e+06,-1.921342e+07,3.771757e+06,-1.911092e+07,-75.697254,-11699.518535
3,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_4,13:25:33,13:43:13,NaN,NaN,NaN,3.773385e+06,-1.911344e+07,3.773131e+06,-1.914465e+07,3960.747482,45397.844769
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,15:03:32,15:10:32,NaN,NaN,NaN,3.773118e+06,-1.914490e+07,NaN,NaN,1853.469522,7094.667696
5,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_0,08:08:23,08:20:08,17.29,17.29,17.29,3.771380e+06,-1.933274e+07,3.769933e+06,-1.934095e+07,-1447.153380,-8212.508976
6,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_6,12:08:23,12:13:24,NaN,NaN,NaN,3.771380e+06,-1.933274e+07,3.769983e+06,-1.934265e+07,-723.576690,-4106.254488
7,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,15:29:09,15:30:56,NaN,NaN,NaN,3.769978e+06,-1.934136e+07,NaN,NaN,-1060.318150,-7010.207910
8,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_1,08:45:05,08:50:33,NaN,NaN,NaN,3.760880e+06,-1.910042e+07,3.759312e+06,-1.908280e+07,-1568.491625,17623.035652
9,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_2,11:09:02,11:09:02,NaN,NaN,NaN,3.755349e+06,-1.916135e+07,3.755349e+06,-1.916135e+07,-784.245813,8811.517826


In [8]:
test2[:10]

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,x_dir,y_dir
0,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,15:03:32,15:10:32,NaN,NaN,NaN,3.773118e+06,-1.914490e+07,NaN,NaN,1853.469522,7094.667696
1,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,15:29:09,15:30:56,NaN,NaN,NaN,3.769978e+06,-1.934136e+07,NaN,NaN,-1060.318150,-7010.207910
2,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_5,15:26:08,15:26:08,NaN,NaN,NaN,3.757468e+06,-1.923860e+07,NaN,NaN,-392.122906,4405.758913
3,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,15:35:18,15:42:05,NaN,NaN,NaN,3.760505e+06,-1.935500e+07,NaN,NaN,0.000000,0.000000
4,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_4,14:54:07,15:05:14,NaN,NaN,NaN,3.766319e+06,-1.917013e+07,NaN,NaN,-1618.167948,27383.412879
5,0006535be25bb52dd06983447880c964_5,traj_0006535be25bb52dd06983447880c964_5_12,15:22:06,15:22:06,NaN,NaN,NaN,3.776037e+06,-1.916993e+07,NaN,NaN,0.000000,0.000000
6,0006f84bb33ec929d1cda7686f861d0a_31,traj_0006f84bb33ec929d1cda7686f861d0a_31_3,15:00:12,15:00:12,NaN,NaN,NaN,3.760009e+06,-1.923361e+07,NaN,NaN,0.974046,77.083372
7,00093ae562586aed0e053b8431e8ace4_23,traj_00093ae562586aed0e053b8431e8ace4_23_10,15:53:09,15:53:09,NaN,NaN,NaN,3.756665e+06,-1.914692e+07,NaN,NaN,0.000000,0.000000
8,000c739e444a70e1804d757a0580caaa_31,traj_000c739e444a70e1804d757a0580caaa_31_3,15:41:31,15:41:31,NaN,NaN,NaN,3.768082e+06,-1.922134e+07,NaN,NaN,-1224.514399,-33654.583531
9,000d479078af08618bddc7f09082b8c3_11,traj_000d479078af08618bddc7f09082b8c3_11_6,14:58:06,15:03:40,NaN,NaN,NaN,3.766505e+06,-1.920176e+07,NaN,NaN,11434.807669,58770.028287


In [9]:
def within_measure(x, y):
    #  3750901.5068 ≤ 𝑥 ≤ 3770901.5068
    #  −19268905.6133 ≤ 𝑦 ≤ −19208905.6133
    if 3750901.5068 <= x and x <= 3770901.5068 and -19268905.6133 <= y and y <= -19208905.6133:
        return 1
    else:
        return 0

def time_to_sec(values):
    return pd.to_timedelta(values).dt.total_seconds().astype(int)

train.time_entry = time_to_sec(train.time_entry)
train.time_exit = time_to_sec(train.time_exit)
test.time_entry = time_to_sec(test.time_entry)
test.time_exit = time_to_sec(test.time_exit)
test2.time_entry = time_to_sec(test.time_entry)
test2.time_exit = time_to_sec(test.time_exit)


In [10]:
"""
X = train['x_exit'].values 
Y = train['y_exit'].values
train.drop(columns=['x_exit','y_exit'],inplace=True)
test = test2
test.drop(columns=['x_exit','y_exit'],inplace=True)

city = [within_measure(x, y) for x,y in zip(X,Y)]
"""
y_train=pd.DataFrame() 
y_train['x_exit']= train['x_exit']
y_train['y_exit']= train['y_exit']

train.drop(columns=['x_exit','y_exit'],inplace=True)
test2.drop(columns=['x_exit','y_exit'],inplace=True)

In [11]:

for col in ['time_entry','time_exit','vmax','vmin','vmean','x_entry','y_entry','x_dir','y_dir']:
    mode = train[col].mode()[0]
    test[col].fillna(mode, inplace=True)
    test2[col].fillna(mode, inplace=True)
    train[col].fillna(mode, inplace=True)

In [12]:
from sklearn.neural_network import MLPRegressor


x_train = train.drop(columns=['hash','trajectory_id'])
x_test = test2.drop(columns=['hash','trajectory_id'])

model = MLPRegressor(hidden_layer_sizes=[100],
                      learning_rate_init=.01,
                      max_iter=200,
                      learning_rate='constant',
                      solver='adam',
                      activation='relu',
                     verbose=True)
model.fit(x_train, y_train)


Iteration 1, loss = 10626847921158.86718750
Iteration 2, loss = 8008525926.87770653
Iteration 3, loss = 393850700.77476758
Iteration 4, loss = 386573093.62681168
Iteration 5, loss = 386376536.34027547
Iteration 6, loss = 381498266.33185196
Iteration 7, loss = 380240259.65955329
Iteration 8, loss = 379047646.52453667
Iteration 9, loss = 377021941.70407063
Iteration 10, loss = 373144256.35597867
Iteration 11, loss = 372312554.91072655
Iteration 12, loss = 369521539.83082712
Iteration 13, loss = 367976963.89108366
Iteration 14, loss = 366998170.36018229
Iteration 15, loss = 364533059.44522017
Iteration 16, loss = 360370706.12167662
Iteration 17, loss = 361310118.03526461
Iteration 18, loss = 359360323.31062078
Iteration 19, loss = 356934655.75666475
Iteration 20, loss = 369055218.95059341
Iteration 21, loss = 357722746.79358786
Iteration 22, loss = 365849955.32627416
Iteration 23, loss = 356405782.62661999
Iteration 24, loss = 351241660.41817880
Iteration 25, loss = 354898053.69232434
Ite

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=[100], learning_rate='constant',
       learning_rate_init=0.01, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)

In [13]:
pred = model.predict(x_test)
#pred = pd.DataFrame(data=pred,columns=['x_exit','y_exit'])
pred = pd.DataFrame({'x_exit':pred[:,0],'y_exit':pred[:,1]})

In [14]:
X = pred['x_exit'].values 
Y = pred['y_exit'].values

city = [within_measure(x, y) for x,y in zip(X,Y)]

In [15]:
submission= pd.DataFrame()
submission['id']=trajectory_id 
submission['target'] = city
submission.to_csv('./output/submission.csv',index=False)    
submission[:20]

,id,target
0,traj_00032f51796fd5437b238e3a9823d13d_31_5,0
1,traj_000479418b5561ab694a2870cc04fd43_25_10,0
2,traj_000506a39775e5bca661ac80e3f466eb_29_5,1
3,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,0
4,traj_00063a4f6c12e1e4de7d876580620667_3_4,0
5,traj_0006535be25bb52dd06983447880c964_5_12,0
6,traj_0006f84bb33ec929d1cda7686f861d0a_31_3,1
7,traj_00093ae562586aed0e053b8431e8ace4_23_10,0
8,traj_000c739e444a70e1804d757a0580caaa_31_3,1
9,traj_000d479078af08618bddc7f09082b8c3_11_6,0


In [16]:
min(city)

0

In [17]:
max(city)

1